# Workflow example to generate timelapses

This example illustrates how to use `eo-learn` in order to make a cool timelapse! 

We are going to define `EOTask`s, chain them together into an `EOWorkflow` and then fill the data in a container called `EOPatch`.

## Imports

Here we will load all the packages that we will need for this tutorial

In [ ]:
import os
import imageio
import numpy as np
import datetime
from IPython.display import Image

from sentinelhub import BBox, CRS, MimeType, CustomUrlParam

from eolearn.mask import AddCloudMaskTask, get_s2_pixel_cloud_detector
from eolearn.core import EOPatch, EOTask, FeatureType, LinearWorkflow, SaveToDisk, OverwritePermission, LinearWorkflow, EOWorkflow, LoadFromDisk
from eolearn.features import SimpleFilterTask
from eolearn.io import S2L1CWCSInput
from eolearn.coregistration import ThunderRegistration

## Simple Approach <a id="simple"></a>

First let's create a simple `EOWorkflow`, where we just download the data, produce the timelapse, and then save the data for later usage. We will get to know how to load existing tasks and use them, and you even get a chance to complete parts of the code (*spoilers at the bottom*).

In [ ]:
# Define the locations
data_dir = os.path.join('.', '..', 'outputs', 'timelapse_data')
timelapse_dir = os.path.join('.', '..', 'outputs')

for path in [timelapse_dir, data_dir]:
    if not os.path.isdir(path):
        os.mkdir(path)

We are going to make a timelapse of the construction of the solar farm in Ouarzazate, Morocco. Below you can find the predefined coordinates of the bounding box around the area that we are interested in. Later on, you can also change these values to make a timelapse of your own!

In [ ]:
# time range that we are interested in
time_interval = ('2016-06-01', '2017-01-01')

# bbox of the region of interest in the WGS84 coordinate system
sf_bbox = BBox(bbox=[-6.884329, 31.072926, -6.856161, 31.050254], crs=CRS.WGS84)

### Tasks

The tasks for downloading and saving data are already available in the `eo-learn` package, let's set them up

In [ ]:
# task for downloading Sentinel-2 L1C data (all 13 bands) at 10 m resolution
download_task = S2L1CWCSInput(layer='BANDS-S2-L1C', 
                          resx='10m',
                          resy='10m')

# task for saving data, so we can load it from disk in the upcoming cases
save_task = SaveToDisk(folder=data_dir, 
                       overwrite_permission = OverwritePermission.OVERWRITE_PATCH)

But the EOTask for creating the timelapse is missing, so we have to write it up. Let's do that here.
Below you can find the skeleton of an EOTask, and below that, an incomplete implementation of the TimeLapseTask, try to complete it so it works!

```python
class FooTask(EOTask):
    def __init__(self, foo_params):
        self.foo_params = foo_params

    def execute(self, eopatch, *, runtime_params):
        # do what foo does on input eopatch and return it
        return eopatch
```

In [ ]:
### CODE CELL 1
### SOLUTIONS AT THE BOTTOM

class TimeLapseTask(EOTask):
    def __init__(self, project_dir='.', filename='timelapse.gif', fps=5, brightness_factor=1):
        self.project_dir = project_dir
        self.filename = filename
        self.fps = fps
        self.brightness_factor = brightness_factor

    def execute(self, eopatch):
        
        # define the gif writer
        with imageio.get_writer(os.path.join(self.project_dir, self.filename), mode='I', fps=self.fps) as writer:
            
            # access the data in the EOPatch DATA feature
            for image in ???:
                
                # Sentinel-2 bands for RGB are at positions 3, 2 and 1
                rgb_image = ???
                
                # They are in values of float, let's add a brightness factor and 
                # then clip it & cast the image as uint8 (this is what the writer accepts)
                size_of_uint8 = 255 #(2^8)
                rgb_image = np.array(np.clip(rgb_image*self.brightness_factor, 0, 1) * size_of_uint8, dtype=np.uint8)
                
                # append each acquired image to the gif
                writer.append_data(rgb_image)
        
        # return the original eopatch for other possible tasks
        return eopatch
    

timelapse_task = TimeLapseTask(timelapse_dir, 'timelapse_simple.gif', fps=6, brightness_factor=3)

### Workflow

Now let's put all the tasks into a workflow and execute it

In [ ]:
# CODE CELL 2

# define the workflow
workflow = LinearWorkflow(
    ???
)

# define the extra parameters
execution_args = {
    download_task:{'bbox': sf_bbox, 'time_interval': time_interval},
    save_task: {'eopatch_folder': 'eopatch'}
}

# execute the workflow
result = workflow.execute(execution_args)

What does the resulting linear workflow look like?

In [ ]:
workflow.dependency_graph()

What does the EOPatch look like? 

In [ ]:
eopatch = result.eopatch()
eopatch

What does the produced timelapse look like?

In [ ]:
Image(filename='{}/{}'.format(timelapse_dir, 'timelapse_simple.gif'), width = 500)

If only we could do something about those clouds!!!

Don't worry, that's exactly what we aim to do in the next part, and let's sprinkle some complexity on it!

## Advanced Approach <a id="advanced"></a>

As you can see, clouds are a frequent problem in Earth observation data, so let's try to filter them out using our own cloud detector!

Additionally, the images in the animation can be quite wobbly, one way to improve the animation would be to apply coregistration to all the images, in order to improve the animation stability

We will do all of that, and we will use another nice functionality of the EOWorkflow, which is the fact, that the workflow itself needn't be linear, but can be split into several branches!

To showcase this functionality, let's build a workflow that:
* produces the same simple animation as above
* splits and produces another animation with removed cloudy scenes
* another one which slipts again and applies coregistration to the frames

In the end we can then compare all of these animations and see the true power or `eo-learn`!

### Tasks

The first step is to change the download task with the loading task, since we already have the necessary data. Seems simple enough, right?

In [ ]:
# define the loading task
load_task = LoadFromDisk(data_dir)

Next up we have the cloud masking and cloud filtering tasks, for this we use our standalone package 's2cloudless'. We set it up like this

In [ ]:
# get the cloud classifier
cloud_classifier = get_s2_pixel_cloud_detector(all_bands=True)

# define the cloud masking task at 60 m resolution.
# It saves the cloud mask (boolean) into the MASK feature named 'clm' and
# the cloud probabilities (float) into the DATA feature named 'clp'.
cloud_task = AddCloudMaskTask(cloud_classifier, 
                              'BANDS-S2-L1C', 
                              cm_size_y='60m',
                              cm_size_x='60m', 
                              cmask_feature='clm', 
                              cprobs_feature='clp')

The filter task is a generic task which accepts a filter and some threshold value, it then removes the frames which do not satisfy the given condition.

Let's define the filter class, which is then fed into the filter task

In [ ]:
# CLOUD CELL 3

# define the filter class
class CloudCoverageFilter:
    
    # the init function of the filter accepts a 
    # thresholding value and remembers it
    def __init__(self, max_cc):
        self.max_cc = max_cc

    # the call function of the filter then decides
    # which frame to keep
    def __call__(self, cloud_mask):
        
        # obtain dimensions
        height, width, _ = cloud_mask.shape
        
        # calculate cloud coverage
        cloud_coverage = ???
        
        # return if frame passes condition or not
        return ???
    
# initialize the filter with a 5 % cloud coverage threshold
cloud_filter = CloudCoverageFilter(max_cc=0.05)

# define the filter task and feed the filter into the task
filter_task = SimpleFilterTask((FeatureType.MASK, 'clm'), cloud_filter)

We also need the coregistration task, which also already exists in the `eo-learn` package!

We will use the simple Thunder algorithm, more info [here](http://docs.thunder-project.org/tutorial-registration).

In [ ]:
# define the coregistration task
coregistration_task = ThunderRegistration((FeatureType.DATA, 'BANDS-S2-L1C'), channel=2)

Lastly, let's define all the timelapse tasks for all the cases

In [ ]:
timelapse_task_simple = TimeLapseTask(timelapse_dir, 'timelapse_simple.gif', fps=6, brightness_factor=3)
timelapse_task_filtered = TimeLapseTask(timelapse_dir, 'timelapse_filtered.gif', fps=6, brightness_factor=3)
timelapse_task_filtered_coregistered = TimeLapseTask(timelapse_dir, 'timelapse_filtered_coregistered.gif', fps=6, brightness_factor=3)

Now we can finally create a more complex workflow! You can do this using dependencies in the following sense:

```python
workflow = EOWorkflow(
    ...,
    (task1, [dependency_task], 'Task function'), 
    ...
)
```

In [ ]:
# CODE CELL 4

# define list of tasks and dependencies
task_list = [
    (load_task, [], 'Load data'),
    (timelapse_task_simple, [load_task], 'Simple timelapse'),
    ???, # tuple for the cloud task that depends on the load task
    (filter_task, [cloud_task], 'Filter cloud data'),
    (timelapse_task_filtered, [filter_task], 'Filtered timelapse'),
    ???, # coregistration task that depends on the filter task
    (timelapse_task_filtered_coregistered, [coregistration_task], 'Filtered and coregistered timelapse'),
]

# define workflow
advanced_workflow = EOWorkflow(task_list)

# define the extra parameters
execution_args = {
    load_task: {'eopatch_folder': 'eopatch'}
}

# execute the workflow
result = advanced_workflow.execute(execution_args)

What does the advanced workflow look like in this case?

In [ ]:
advanced_workflow.dependency_graph()

Fancy!!

And the produced timelapses?

In [ ]:
Image(filename='{}/{}'.format(timelapse_dir, 'timelapse_simple.gif'), width=500)

In [ ]:
Image(filename='{}/{}'.format(timelapse_dir, 'timelapse_filtered.gif'), width=500)

In [ ]:
Image(filename='{}/{}'.format(timelapse_dir, 'timelapse_filtered_coregistered.gif'), width=500)

&nbsp; 



&nbsp; 



&nbsp; 



### Code solutions below!



&nbsp; 



&nbsp; 



&nbsp; 


In [ ]:
### CODE SOLUTION CELL 1

class TimeLapseTask(EOTask):
    def __init__(self, project_dir='.', filename='timelapse.gif', fps=5, brightness_factor=1):
        self.project_dir = project_dir
        self.filename = filename
        self.fps = fps
        self.brightness_factor = brightness_factor

    def execute(self, eopatch):
        
        # define the gif writer
        with imageio.get_writer(os.path.join(self.project_dir, self.filename), mode='I', fps=self.fps) as writer:
            
            # access the data in the EOPatch DATA feature
            for image in eopatch.data['BANDS-S2-L1C']:
                
                # Sentinel-2 bands for RGB are at positions 3, 2 and 1
                rgb_image = image[..., [3, 2, 1]]
                
                # They are in values of float, let's add a brightness factor and 
                # then clip it & cast the image as uint8 (this is what the writer accepts)
                size_of_uint8 = 255 #(2^8)
                rgb_image = np.array(np.clip(rgb_image*self.brightness_factor, 0, 1) * size_of_uint8, dtype=np.uint8)
                
                # append each acquired image to the gif
                writer.append_data(rgb_image)
        
        # return the original eopatch for other possible tasks
        return eopatch
    

timelapse_task = TimeLapseTask(timelapse_dir, 'timelapse_simple.gif', fps=6, brightness_factor=3)

In [ ]:
# CODE SOLUTION CELL 2

# define the workflow
workflow = LinearWorkflow(
    download_task,
    timelapse_task,
    save_task
)

# define the extra parameters
execution_args = {
    download_task:{'bbox': sf_bbox, 'time_interval': time_interval},
    save_task: {'eopatch_folder': 'eopatch'}
}

# execute the workflow
result = workflow.execute(execution_args)

In [ ]:
# CLOUD SOLUTION CELL 3

# define the filter class
class CloudCoverageFilter:
    
    # the init function of the filter accepts a 
    # thresholding value and remembers it
    def __init__(self, max_cc):
        self.max_cc = max_cc

    # the call function of the filter then decides
    # which frame to keep
    def __call__(self, cloud_mask):
        
        # obtain dimensions
        height, width, _ = cloud_mask.shape
        
        # calculate cloud coverage
        cloud_coverage = np.sum(cloud_mask) / (height * width)
        
        # return if frame passes condition or not
        return cloud_coverage <= self.max_cc
    
# initialize the filter with a 5 % cloud coverage threshold
cloud_filter = CloudCoverageFilter(max_cc=0.05)

# define the filter task and feed the filter into the task
filter_task = SimpleFilterTask((FeatureType.MASK, 'clm'), cloud_filter)

In [ ]:
# CODE SOLUTION CELL 4

# define list of tasks and dependencies
task_list = [
    (load_task, [], 'Load data'),
    (timelapse_task_simple, [load_task], 'Simple timelapse'),
    (cloud_task, [load_task], 'Add cloud data'),
    (filter_task, [cloud_task], 'Filter cloud data'),
    (timelapse_task_filtered, [filter_task], 'Filtered timelapse'),
    (coregistration_task, [filter_task], 'Coregister frames'),
    (timelapse_task_filtered_coregistered, [coregistration_task], 'Filtered and coregistered timelapse'),
]

# define workflow
advanced_workflow = EOWorkflow(task_list)

# define the extra parameters
execution_args = {
    load_task: {'eopatch_folder': 'eopatch'}
}

# execute the workflow
result = advanced_workflow.execute(execution_args)